In [1]:
from pymongo import MongoClient

In [14]:
client = MongoClient('localhost', 27017)

In [15]:
db = client.publicru_test

In [16]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'publicru_test')

In [17]:
collection = db.documents_collection

In [18]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'publicru_test'), 'documents_collection')

In [19]:
import datetime
post = {"author": "Mike",
        "text": "My first blog post!",
        "tags": ["mongodb", "python", "pymongo"],
        "date": datetime.datetime.utcnow()}

In [20]:
collection.find()

In [22]:
post_id = collection.insert_one(post).inserted_id

In [23]:
post_id

ObjectId('5e778fe10e27f8187f4b9540')

In [24]:
db.list_collection_names()

['documents_collection']

In [25]:
for i in collection.find():
    print(i)

{'_id': ObjectId('5e778fe10e27f8187f4b9540'), 'author': 'Mike', 'text': 'My first blog post!', 'tags': ['mongodb', 'python', 'pymongo'], 'date': datetime.datetime(2020, 3, 22, 16, 17, 11, 463000)}


In [26]:
collection.find_one()

{'_id': ObjectId('5e778fe10e27f8187f4b9540'),
 'author': 'Mike',
 'text': 'My first blog post!',
 'tags': ['mongodb', 'python', 'pymongo'],
 'date': datetime.datetime(2020, 3, 22, 16, 17, 11, 463000)}

In [28]:
collection.find_one({"author": "Mike"})

{'_id': ObjectId('5e778fe10e27f8187f4b9540'),
 'author': 'Mike',
 'text': 'My first blog post!',
 'tags': ['mongodb', 'python', 'pymongo'],
 'date': datetime.datetime(2020, 3, 22, 16, 17, 11, 463000)}

In [29]:
collection.find_one({"author": "Eliot"})

In [44]:
new_posts = [{"author": "Mike",
              "text": "Another post!",
              "tags": ["bulk", "insert"],
              "date": datetime.datetime(2009, 11, 12, 11, 14)},
             {"author": "Eliot",
              "title": "MongoDB is fun",
              "text": "and pretty easy too!",
              "date": datetime.datetime(2009, 11, 10, 10, 45)}]
result = collection.insert_many(new_posts)
result.inserted_ids

[ObjectId('5e7792970e27f8187f4b9541'), ObjectId('5e7792970e27f8187f4b9542')]

In [45]:
collection.find_one({"author": "Eliot"})

{'_id': ObjectId('5e7792970e27f8187f4b9542'),
 'author': 'Eliot',
 'title': 'MongoDB is fun',
 'text': 'and pretty easy too!',
 'date': datetime.datetime(2009, 11, 10, 10, 45)}

In [46]:
collection.find_one({"author": "Mike"})

{'_id': ObjectId('5e778fe10e27f8187f4b9540'),
 'author': 'Mike',
 'text': 'My first blog post!',
 'tags': ['mongodb', 'python', 'pymongo'],
 'date': datetime.datetime(2020, 3, 22, 16, 17, 11, 463000)}

In [48]:
for i in collection.find():
    print(i["_id"])

5e778fe10e27f8187f4b9540
5e7792970e27f8187f4b9541
5e7792970e27f8187f4b9542


In [57]:
collection.count_documents({})

3

In [58]:
collection.count_documents({"author": "Mike"})

2

In [59]:
collection.index_information()

{'_id_': {'v': 2,
  'key': [('_id', 1)],
  'ns': 'publicru_test.documents_collection'}}

In [60]:
result = collection.delete_many({"author": "Mike"})
result.deleted_count

2

In [61]:
collection.count_documents({})

1

In [62]:
collection.delete_many({})

In [63]:
collection.count_documents({})

0

In [30]:
import gensim
import pandas as pd
import numpy as np
import os
import artm
import logging
import glob
import re
from gensim.models.phrases import Phrases, Phraser
from gensim.models.word2vec import Text8Corpus
from gensim.test.utils import datapath
from sklearn.neighbors import NearestNeighbors
import pickle
from pathlib import Path
from tqdm.notebook import tqdm

In [31]:
raw_data_path = os.path.join("..", "..", "data", "raw")
processed_data_path = os.path.join("..", "..", "data", "processed")
models_path = os.path.join("..", "..", "models")
experiments_path = os.path.join("..", "..", "experiments")

In [36]:
pd.options.display.max_rows = 999

In [168]:
df_processed_documents =  pd.read_pickle(os.path.join(processed_data_path, "publicru-dataset-2020-03-16", "processed_documents.pkl"))

In [171]:
df_processed_documents["id"] = df_processed_documents.index

In [172]:
df_processed_documents.head()

,title,annotation,body,authors,issue_id,edition_id,issue_date,id,edition_type,edition_periodicity,edition_description,edition_name,t_title,t_annotation,t_edition_name,t_edition_description,t_body
id,,,,,,,,,,,,,,,,,
325,Эффект слабого рубля,Мы взяли две компании с сопоставимой выручкой ...,<body><p><b>Мы взяли две компании с сопоставим...,NaN,9,1,2016-10-01,325,magazine,monthly,"Журнал «РБК» — это аналитические статьи, посвя...",РБК Журнал,"[эффект, слабый, рубль]","[взять, компания, сопоставимый, выручка, оцени...",рбк_журнал,"[журнал, рбк, аналитический, статья, проблема,...","[взять, компания, сопоставимый, выручка, оцени..."
21225,РОССТАТ,Индексы потребительских цен на товары и услуги...,<body><p><b>Индексы потребительских цен на тов...,NaN,599,15,2016-07-18,21225,newspaper,daily,«Российская газета» - издание нового российско...,Российская газета,[росстат],"[индекс, потребительский, цена, товар, услуга,...",российская_газета,"[российский, газета, издание, новое, российски...","[индекс, потребительский, цена, товар, услуга,..."
21226,Неотложка с планшетом,ЗДОРОВЬЕ . В системе ЕМИАС зарегистрировались ...,<body><p><b>ЗДОРОВЬЕ</b> </p>\n<p>В системе ЕМ...,Сергей Жуков,599,15,2016-07-18,21226,newspaper,daily,«Российская газета» - издание нового российско...,Российская газета,"[неотложка, планшет]","[здоровье, система, емиас, зарегистрироваться,...",российская_газета,"[российский, газета, издание, новое, российски...","[здоровье, система, емиас, зарегистрироваться,..."
1638,ПРОГНОЗЫ НОМЕРА,К 2017-2018 годам ныне растущий агрокомплекс с...,<body><p>К 2017-2018 годам ныне растущий агрок...,NaN,42,2,2016-10-03,1638,magazine,weekly,Журнал «Эксперт» — одно из самых влиятельных е...,Эксперт,"[прогноз, номер]","[год, ныне, агрокомплекс, столкнуться, целое, ...",эксперт,"[журнал, эксперт, самый, влиятельный, еженедел...","[год, ныне, агрокомплекс, столкнуться, целое, ..."
2,ТАНЕЦ НЕНАСТОЯЩЕГО ЛЕБЕДЯ,ПЕРЕД НАМИ КАК БУДТО СЦЕНА ИЗ БАЛЕТА: ДВА МИСТ...,<body><p>ПЕРЕД НАМИ КАК БУДТО СЦЕНА ИЗ БАЛЕТА:...,NaN,1,1,2016-01-01,2,magazine,monthly,"Журнал «РБК» — это аналитические статьи, посвя...",РБК Журнал,"[танец, ненастоящий, лебедь]","[сцена, балет, мистический, злодей, собираться...",рбк_журнал,"[журнал, рбк, аналитический, статья, проблема,...","[сцена, балет, мистический, злодей, собираться..."


In [37]:
df_processed_documents.t_edition_name.value_counts()

российская_газета                                                                                     57906
коммерсантъ                                                                                           49215
московский_комсомолец                                                                                 44015
комсомольская_правда                                                                                  37439
независимая_газета                                                                                    34487
ведомости                                                                                             25199
известия                                                                                              22046
деловой_петербург                                                                                     13689
новая_газета                                                                                          11870
рбк_газета                  

In [174]:
df1 = df_processed_documents[df_processed_documents.t_edition_name == "российский_налоговый_курьер"]

In [175]:
df2 = df_processed_documents[df_processed_documents.t_edition_name == "управление_персоналом"]

In [176]:
df1.head()

,title,annotation,body,authors,issue_id,edition_id,issue_date,id,edition_type,edition_periodicity,edition_description,edition_name,t_title,t_annotation,t_edition_name,t_edition_description,t_body
id,,,,,,,,,,,,,,,,,
545296,"Ошибка бухгалтера, из-за которой компания поте...",Проблемные реквизиты в платежке. Опасные форму...,<p><b>Проблемные реквизиты в платежке</b></p>\...,"Вера Лозинская, эксперт журнала «Российский на...",9251,104,2018-10-01,545296,magazine,monthly,Специализированный практический журнал для гла...,Российский налоговый курьер,"[ошибка, бухгалтер, компания, потерять, деньга]","[проблемный, реквизит, платежка, опасный, форм...",российский_налоговый_курьер,"[практический, журнал, главный, бухгалтер, ауд...","[проблемный, реквизит, платежка, опасный, форм..."
545297,Даты приказов и актов могут стать причиной штр...,С 1 октября оспорить штраф за «забытых» застра...,<p>С 1 октября оспорить штраф за «забытых» зас...,NaN,9251,104,2018-10-01,545297,magazine,monthly,Специализированный практический журнал для гла...,Российский налоговый курьер,"[дата, приказ, акт, стать, причина, штраф]","[октябрь, оспорить, штраф, забытый, труд, удат...",российский_налоговый_курьер,"[практический, журнал, главный, бухгалтер, ауд...","[октябрь, оспорить, штраф, забытый, труд, удат..."
255097,"Три норматива, которые выгодно пересмотреть пр...",Почему выгодны сверхнормативные суточные. Как ...,<p><b>Почему выгодны сверхнормативные суточные...,Дмитрий Святский,4479,104,2017-11-01,255097,magazine,monthly,Специализированный практический журнал для гла...,Российский налоговый курьер,"[норматив, выгодно, пересмотреть, прямо]","[выгодный, сверхнормативный, суточный, пересма...",российский_налоговый_курьер,"[практический, журнал, главный, бухгалтер, ауд...","[выгодный, сверхнормативный, суточный, пересма..."
255098,Четыре повода быстрее самортизировать актив,Чаще всего используют коэффициент по предметам...,<p><b>Чаще всего используют коэффициент по пре...,Алексей Головлев,4479,104,2017-11-01,255098,magazine,monthly,Специализированный практический журнал для гла...,Российский налоговый курьер,"[повод, самортизировать, актив]","[использовать, коэффициент, предмет, лизинг, э...",российский_налоговый_курьер,"[практический, журнал, главный, бухгалтер, ауд...","[использовать, коэффициент, предмет, лизинг, э..."
255099,"Минфин уточнил, когда признавать доходом преми...",Какие премии признавать как зарплату. Как отра...,<p><b>Какие премии признавать как зарплату</b>...,Инна Михайлова,4479,104,2017-11-01,255099,magazine,monthly,Специализированный практический журнал для гла...,Российский налоговый курьер,"[минфин, уточнить, признавать, доход, премия, ...","[премия, признавать, зарплата, отражать, ндфл,...",российский_налоговый_курьер,"[практический, журнал, главный, бухгалтер, ауд...","[премия, признавать, зарплата, отражать, ндфл,..."


In [177]:
df1.columns.tolist()

['title',
 'annotation',
 'body',
 'authors',
 'issue_id',
 'edition_id',
 'issue_date',
 'id',
 'edition_type',
 'edition_periodicity',
 'edition_description',
 'edition_name',
 't_title',
 't_annotation',
 't_edition_name',
 't_edition_description',
 't_body']

In [178]:
client = MongoClient('localhost', 27017)

In [179]:
db = client.publicru_test

In [181]:
collection = db.documents_collection

In [182]:
posts = [ {"doc_id": row["id"],
   "title": row["title"],
   "body": row["body"],
   "annotation": row["annotation"],
   "issue_id": row["issue_id"],
   "edition_id": row["edition_id"],
   "annotation": row["annotation"],
   "t_title": row["t_title"],
   "t_edition_name": row["t_edition_name"],
   "issue_date": row["issue_date"],
   "t_body": row["t_body"]}  for _, row in df1.iterrows()]
collection.insert_many(posts)

In [131]:
result = collection.insert_many(posts)

In [132]:
collection.count_documents({})

1530

In [183]:
posts = [ {
    "doc_id": row["id"],
    "title": row["title"],
    "body": row["body"],
    "annotation": row["annotation"],
    "issue_id": row["issue_id"],
    "edition_id": row["edition_id"],
    "annotation": row["annotation"],
    "t_title": row["t_title"],
    "t_edition_name": row["t_edition_name"],
    "issue_date": row["issue_date"],
    "t_body": row["t_body"]
}  for _, row in df2.iterrows()]
collection.insert_many(posts)

In [184]:
collection.count_documents({})

2478

In [180]:
# result = collection.delete_many({})
# result.deleted_count

948

In [185]:
collection.count_documents({"t_edition_name": "российский_налоговый_курьер"})

1530

In [186]:
collection.find_one({"doc_id": 1638})

In [95]:
result

In [155]:
collection.find_one()

{'_id': ObjectId('5e77e6c40e27f8187f4ba8a0'),
 'doc_id': 245,
 'title': 'Автоматизируйте подбор персонала',
 'body': '<p><b>Тесты и анкеты - фильтр для тех кто не мотивирован на работу.</b></p>\n<p>Алексей Стоякин, Консалтинговая компания МДД Групп</p>\n<p><img alt="" src="494059/1.jpg" title=""/></p>\n<p><b>[УП] Если компании нужны продажники "незримого продукта", эфимерного (услуга типа треннинг, пиар, консалтинг и т.д.), то как стоит описать вакансию?</b></p>\n<p>- Сама структура вакансии в общем-то ничем не будет отличаться от вакансии на любой другой продукт. Работодатель всегда должен помнить, что вакансия - это то же рекламное объявление. Составите ее скучно, как у всех - отклик у вас будет соответствующий. Напишите, что работать у вас очень просто и делать особо ничего не потребуется - получите толпы халявщиков, стоящих под вашей дверью.</p>\n<p>В вакансии вам нужно описать то, чем ваша компания занимается, чем вы выделяетесь на рынке среди прочих. Как и в любой рекламе нужно о

In [110]:
find = collection.find({"t_edition_name": "российский_налоговый_курьер"})

In [112]:
next(find)['text']

'Даты приказов и актов могут стать причиной штрафа по СЗВ-М'

In [113]:
next(find)['text']

'Три норматива, которые выгодно пересмотреть прямо сейчас'

In [114]:
next(find)['text']

'Четыре повода быстрее самортизировать актив'

In [189]:
for x in collection.find({"t_edition_name": "российский_налоговый_курьер"}, { "doc_id": 1, "issue_id": 1, "edition_id": 1, "title": 1, }):
    print(x["doc_id"], x["issue_id"], x["edition_id"], x["title"])

545296 9251 104 Ошибка бухгалтера, из-за которой компания потеряет деньги
545297 9251 104 Даты приказов и актов могут стать причиной штрафа по СЗВ-М
255097 4479 104 Три норматива, которые выгодно пересмотреть прямо сейчас
255098 4479 104 Четыре повода быстрее самортизировать актив
255099 4479 104 Минфин уточнил, когда признавать доходом премии за работу
255100 4479 104 Что сейчас налоговикам не нравится в учете расходов на командировки
255101 4479 104 ФНС рассказала, как подтвердить вычет
255102 4479 104 Советы чиновников, как по-новому заполнять счета-фактуры
255103 4479 104 Новые рекомендации по заполнению РСВ
255104 4479 104 По каким правилам начислят пени на старые долги
255105 4479 104 ФНС разъяснила, как оформлять чек на услугу
255106 4479 104 Получить вычет на обучение ребенка стало проще
255107 4479 104 Чиновники требуют взносы с социальных выплат
255108 4479 104 Появились банкноты в 200 и 2000 рублей
255109 4479 104 Известны лимиты по взносам на следующий год
255110 4479 104 К

In [187]:
for x in collection.find({"t_edition_name": "управление_персоналом"}, { "doc_id": 1, "issue_id": 1, "edition_id": 1, "title": 1, }):
    print(x["doc_id"], x["issue_id"], x["edition_id"], x["title"])

494059 8358 245 Автоматизируйте подбор персонала
494060 8358 245 Тренинги очень полезны бизнесу
494061 8358 245 КОЛОНКА РЕДАКТОРА
494062 8358 245 Бизнес против правил: Как Андрей Трубников создал Natura Siberica и захватил рынок органической косметики в России
494063 8358 245 Если у тренера есть ХАРИЗМА, то люди идут на его тренинги
494064 8358 245 Комментарии к статье «Разбираться в людях - это просто. Не верите? Попробуйте»
494065 8358 245 Фрилансеры - свободный народ????!!!!
494066 8358 245 Тренинг должен приносить пользу компании
494067 8358 245 Японский опыт "кружков качества" можно эффективно применять и у нас
494068 8358 245 Несколько вопросов кандидату на «топ» позицию на понимание основ «управления (менеджмента)»
494069 8358 245 Печень и желудок не должны конфликтовать
520920 8973 245 КОЛОНКА РЕДАКТОРА
581181 10300 245 КОЛОНКА РЕДАКТОРА
581182 10300 245 Лидер обязан быть хитрее и мудрее провокаций
581183 10300 245 Дмитрий Портнягин преуспел. Попробуйте и вы!!!
581184 10300 245

In [126]:
next(collection.find({"t_edition_name": "российский_налоговый_курьер"}, { "_id": 1, "doc_id": 1,  }))

{'_id': ObjectId('5e77d77c0e27f8187f4b9544'), 'doc_id': 104}

In [218]:
db.command("collstats", "documents_collection", scale=1024*1024)["storageSize"]

29

In [208]:
db.command("dbstats", scale=1024*1024)

{'db': 'publicru_test',
 'collections': 1,
 'views': 0,
 'objects': 2478,
 'avgObjSize': 24281.251412429378,
 'dataSize': 57.38157367706299,
 'storageSize': 29.30859375,
 'numExtents': 0,
 'indexes': 1,
 'indexSize': 0.0625,
 'fsUsedSize': 359482.1640625,
 'fsTotalSize': 937871.015625,
 'ok': 1.0}

In [203]:
30732288.0 / 1024 /1024 /1024

0.028621673583984375

In [204]:
collection.command("collstats")

TypeError: 'Collection' object is not callable. If you meant to call the 'command' method on a 'Collection' object it is failing because no such method exists.